*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# The Microsoft Toolkit of Multi-Task Deep Neural Networks for Natural Language Understanding


## Summary

MT-DNN is an open-source natural language understanding (NLU) toolkit that makes it easy for researchers and developers to train customized deep learning models. Built upon PyTorch and Transformers, MT-DNN is designed to facilitate rapid
customization for a broad spectrum of NLU tasks, using a variety of objectives (classification, regression, structured prediction) and text encoders (e.g., RNNs, BERT, RoBERTa, UniLM). A unique feature of MT-DNN is its built-in support for robust and transferable learning using the adversarial multi-task learning paradigm. To enable efficient production deployment, MT-DNN supports multitask knowledge distillation, which can substantially compress a deep neural model without significant performance drop. We demonstrate the effectiveness of MT-DNN on a wide range of NLU applications across general and biomedical domains. The pip installable package and pretrained models will be publicly available at https://github.com/microsoft/mt-dnn.

### Design

MT-DNN is designed for modularity, flexibility, and ease of use. These modules are built upon PyTorch (Paszke et al., 2019) and Transformers (Wolf
et al., 2019), allowing the use of the SOTA pretrained models, e.g., BERT (Devlin et al., 2019), RoBERTa (Liu et al., 2019c) and UniLM (Dong
et al., 2019). The unique attribute of this package is a flexible interface for adversarial multi-task fine-tuning and knowledge distillation, so that researchers and developers can build large SOTA NLU models and then compress them to small ones
for online deployment.The overall workflow and system architecture are shown in figures 1 and 3 respectively.


![Workflow Design](https://nlpbp.blob.core.windows.net/images/mt-dnn2.JPG)

The above figure shows workflow of MT-DNN: train a neural language model on a large amount of unlabeled raw text
to obtain general contextual representations; then finetune the learned contextual representation on downstream tasks, e.g. GLUE (Wang et al., 2018); lastly, distill this large model to a lighter one for online deployment. In the later two phrases, we can leverage powerful multi-task learning and adversarial training to further improve performance.

## Architecture

![overall_arch](https://nlpbp.blob.core.windows.net/images/mt-dnn.png)
The figure above shows the overall system architecture. The lower layers are shared across all tasks while the top layers are taskspecific. The input X (either a sentence or a set of sentences) is first represented as a sequence of embedding
vectors, one for each word, in l1. Then the encoder, e.g a Transformer or recurrent neural network (LSTM) model,
captures the contextual information for each word and generates the shared contextual embedding vectors in l2.
Finally, for each task, additional task-specific layers generate task-specific representations, followed by operations
necessary for classification, similarity scoring, or relevance ranking. In case of adversarial training, we perturb
embeddings from the lexicon encoder and then add an extra loss term during the training. Note that for the
inference phrase, it does not require perturbations.


### Text Classification of MultiNLI Sentences using MT-DNN

This notebook utilizes the pip installable package that implements the Multi-Task Deep Neural Network Toolkit (MTDNN) for Natural Language Understanding. It's recommended to run this notebook on GPU machines as it's very computationally intensive.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
import torch
import shutil

from tempfile import TemporaryDirectory
from utils_nlp.dataset.multinli import download_tsv_files_and_extract

from mtdnn.common.types import EncoderModelType
from mtdnn.configuration_mtdnn import MTDNNConfig
from mtdnn.modeling_mtdnn import MTDNNModel
from mtdnn.process_mtdnn import MTDNNDataProcess
from mtdnn.tasks.config import MTDNNTaskDefs
from mtdnn.data_builder_mtdnn import MTDNNDataBuilder
from mtdnn.tokenizer_mtdnn import MTDNNTokenizer

/home/useradmin/miniconda3/envs/nlp_gpu/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Define Configuration, Tasks and Model Objects

## Introduction
In this notebook, we fine-tune and evaluate MT-DNN models on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.


In [3]:
# Define Configuration, Tasks and Model Objects
ROOT_DIR = TemporaryDirectory().name
OUTPUT_DIR = os.path.join(ROOT_DIR, 'checkpoint')
os.makedirs(OUTPUT_DIR) if not os.path.exists(OUTPUT_DIR) else OUTPUT_DIR

LOG_DIR = os.path.join(ROOT_DIR, 'tensorboard_logdir')
os.makedirs(LOG_DIR) if not os.path.exists(LOG_DIR) else LOG_DIR

DATA_DIR = os.path.join(ROOT_DIR, 'data')
os.makedirs(DATA_DIR) if not os.path.exists(DATA_DIR) else DATA_DIR

DATA_SOURCE_DIR = os.path.join(DATA_DIR, "MNLI")

BATCH_SIZE = 64
NUM_EPOCHS = 5

Exploring the location for our data to be downloaded, model to be checkpointed and logs to be dumped

In [22]:
print(DATA_DIR)
print(OUTPUT_DIR)
print(LOG_DIR)

/tmp/tmpv27d9tk4/data
/tmp/tmpv27d9tk4/checkpoint
/tmp/tmpv27d9tk4/tensorboard_logdir


## Read Dataset
We start by loading a subset of the data. The following function also downloads and extracts the files, if they don't exist in the data folder.

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators. The sentence pairs are also classified into *genres* that allow for more coverage and better evaluation of NLI models.

For our classification task, we use the first sentence only as the text input, and the corresponding genre as the label. We select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

In [5]:
download_tsv_files_and_extract(DATA_DIR)

100%|██████████| 305k/305k [00:04<00:00, 62.6kKB/s] 


Downloaded file to:  /tmp/tmpv27d9tk4/data/MNLI


### Define a Configuration Object 

Create a model configuration object, `MTDNNConfig`, with the necessary parameters to initialize the MT-DNN model. Initialization without any parameters will default to a similar configuration that initializes a BERT model. 

In [6]:
config = MTDNNConfig(batch_size=BATCH_SIZE)


### Create Task Definition Object  

Define the task parameters to train for and initialize an `MTDNNTaskDefs` object. Create a task parameter dictionary. Definition can be a single or multiple tasks to train.  `MTDNNTaskDefs` can take a python dict, yaml or json file with task(s) defintion.

The data source directory is the path of data downloaded and extracted above using `download_tsv_files_and_extract` which is the `MNLI` dir under the `DATA_DIR` temporary directory.    

The data source has options that are set to drive each task pre-processing; `data_process_opts`


In [7]:
tasks_params = {
    "mnli": {
        "data_format": "PremiseAndOneHypothesis",
        "encoder_type": "BERT",
        "dropout_p": 0.3,
        "enable_san": True,
        "labels": ["contradiction", "neutral", "entailment"],
        "metric_meta": ["ACC"],
        "loss": "CeCriterion",
        "kd_loss": "MseCriterion",
        "n_class": 3,
        "split_names": [
            "train",
            "dev_matched",
            "dev_mismatched",
            "test_matched",
            "test_mismatched",
        ],
        "data_source_dir": DATA_SOURCE_DIR,
        "data_process_opts": {"header": True, "is_train": True, "multi_snli": False,},
        "task_type": "Classification",
    },
}

# Define the tasks
task_defs = MTDNNTaskDefs(tasks_params)

06/24/2020 11:56:20 - mtdnn.tasks.config - INFO - Mapping Task attributes
06/24/2020 11:56:20 - mtdnn.tasks.config - INFO - Configured task definitions - ['mnli']



### Create the MTDNN Data Tokenizer Object  

Create a data tokenizing object, `MTDNNTokenizer`. Based on the model initial checkpoint, it wraps around the model's Huggingface transformers library to encode the data to MT-DNN format. This becomes the input to the data building stage.  


In [13]:
tokenizer = MTDNNTokenizer(do_lower_case=True)

#### Testing out the Tokenizer encode function on a sample text
`tokenizer.encode("What NLP toolkit do you recommend", "MT-DNN is a fantastic toolkit")`

In [24]:
print(tokenizer.encode("What NLP toolkit do you recommend", "MT-DNN is a fantastic toolkit"))

([101, 2054, 17953, 2361, 6994, 23615, 2079, 2017, 16755, 102, 11047, 1011, 1040, 10695, 2003, 1037, 10392, 6994, 23615, 102], None, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


## Data Preprocessing

### Create the Data Builder Object  

Create a data preprocessing object, `MTDNNDataBuilder`. This class is responsible for converting the data into the MT-DNN format depending on the task.  
 

Define a data builder that handles the creating of each task's vectorized data utilizing the model tokenizer. This will build out the vectorized data needed for creating the training, test and development PyTorch dataloaders

In [15]:
## Load and build data
data_builder = MTDNNDataBuilder(
    tokenizer=tokenizer,
    task_defs=task_defs,
    data_dir=DATA_DIR,
    canonical_data_suffix="canonical_data",
    dump_rows=True,
)

## Build data to MTDNN Format
## Iterable of each specific task and processed data
vectorized_data = data_builder.vectorize()

06/24/2020 11:58:03 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 392702 samples for mnli at /tmp/tmpv27d9tk4/data/canonical_data_2/mnli_train.tsv
06/24/2020 11:58:03 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9815 samples for mnli at /tmp/tmpv27d9tk4/data/canonical_data_2/mnli_dev_matched.tsv
06/24/2020 11:58:03 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9832 samples for mnli at /tmp/tmpv27d9tk4/data/canonical_data_2/mnli_dev_mismatched.tsv
06/24/2020 11:58:03 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9796 samples for mnli at /tmp/tmpv27d9tk4/data/canonical_data_2/mnli_test_matched.tsv
06/24/2020 11:58:03 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9847 samples for mnli at /tmp/tmpv27d9tk4/data/canonical_data_2/mnli_test_mismatched.tsv
mnli_train
06/24/2020 11:58:03 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI TRAIN' Task


Building Data For Premise and One Hypothesis: 392702it [05:21, 1222.77it/s]

06/25/2020 12:03:26 - mtdnn.data_builder_mtdnn - INFO - Saving data to /tmp/tmpv27d9tk4/data/canonical_data_2/bert_base_uncased/mnli_train.json



Saving Data For PremiseAndOneHypothesis: 100%|██████████| 392702/392702 [00:05<00:00, 74643.01it/s]

mnli_dev_matched
06/25/2020 12:03:32 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI DEV MATCHED' Task


...


Saving Data For PremiseAndOneHypothesis: 100%|██████████| 9796/9796 [00:00<00:00, 67821.14it/s]

mnli_test_mismatched
06/25/2020 12:03:56 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI TEST MISMATCHED' Task



Building Data For Premise and One Hypothesis: 9847it [00:09, 1068.40it/s]

06/25/2020 12:04:05 - mtdnn.data_builder_mtdnn - INFO - Saving data to /tmp/tmpv27d9tk4/data/canonical_data_2/bert_base_uncased/mnli_test_mismatched.json



Saving Data For PremiseAndOneHypothesis: 100%|██████████| 9847/9847 [00:00<00:00, 75038.45it/s]


### Create the Data Processing Object  

Create a data preprocessing object, `MTDNNDataProcess`. This creates the training, test and development PyTorch dataloaders needed for training and testing. We also need to retrieve the necessary training options required to initialize the model correctly, for all tasks.  

Define a data process that handles creating the training, test and development PyTorch dataloaders

In [16]:
# Make the Data Preprocess step and update the config with training data updates
data_processor = MTDNNDataProcess(
    config=config, task_defs=task_defs, vectorized_data=vectorized_data
)

06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Starting to process the training data sets
06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Loading mnli_train as task 0
06/25/2020 12:08:47 - mtdnn.dataset_mtdnn - INFO - Loaded 392702 samples out of 392702
06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Starting to process the testing data sets
06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Loading mnli_dev_matched as task 0
06/25/2020 12:08:47 - mtdnn.dataset_mtdnn - INFO - Loaded 9815 samples out of 9815
06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Loading mnli_dev_mismatched as task 0
06/25/2020 12:08:47 - mtdnn.dataset_mtdnn - INFO - Loaded 9832 samples out of 9832
06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Loading mnli_test_matched as task 0
06/25/2020 12:08:47 - mtdnn.dataset_mtdnn - INFO - Loaded 9796 samples out of 9796
06/25/2020 12:08:47 - mtdnn.process_mtdnn - INFO - Loading mnli_test_mismatched as task 0
06/25/2020 12:08:47 - mtdnn.dataset_mtdnn - I

Retrieve the processed batch multitask batch data loaders for training, development and test

In [17]:
multitask_train_dataloader = data_processor.get_train_dataloader()
dev_dataloaders_list = data_processor.get_dev_dataloaders()
test_dataloaders_list = data_processor.get_test_dataloaders()

Now we can retrieve the training options, from the processor, to initialize model with.

In [18]:
decoder_opts = data_processor.get_decoder_options_list()
task_types = data_processor.get_task_types_list()
dropout_list = data_processor.get_tasks_dropout_prob_list()
loss_types = data_processor.get_loss_types_list()
kd_loss_types = data_processor.get_kd_loss_types_list()
tasks_nclass_list = data_processor.get_task_nclass_list()

Let us update the batch steps

In [19]:
num_all_batches = data_processor.get_num_all_batches()

### Instantiate the MTDNN Model

Now we can go ahead and create an `MTDNNModel` model

In [20]:
model = MTDNNModel(
    config,
    task_defs,
    pretrained_model_name="bert-base-uncased",
    num_train_step=num_all_batches,
    decoder_opts=decoder_opts,
    task_types=task_types,
    dropout_list=dropout_list,
    loss_types=loss_types,
    kd_loss_types=kd_loss_types,
    tasks_nclass_list=tasks_nclass_list,
    multitask_train_dataloader=multitask_train_dataloader,
    dev_dataloaders_list=dev_dataloaders_list,
    test_dataloaders_list=test_dataloaders_list,
    output_dir=OUTPUT_DIR,
    log_dir=LOG_DIR 
)

idx: 0, number of task labels: 3


## Model Finetuning, Prediction and Evaluation

### Fit and finetune model on five epochs and predict using the training and test  

At this point the MT-DNN model allows us to fit to the model and create predictions. The fit takes an optional `epochs` parameter that overwrites the epochs set in the `MTDNNConfig` object. 

In [15]:
model.fit(epochs=NUM_EPOCHS)

### Prediction
Perform inference using the last (best) checkpointed model. With 5 epochs, the last model would be `model_4.pt`

In [ ]:
model.predict(trained_model_chckpt=f"{OUTPUT_DIR}/model_4.pt")

### Evaluation

In [21]:
results = {}
dev_result_files = list(filter(lambda x: x.endswith('.json') and 'dev' in x, os.listdir(OUTPUT_DIR))) 
for d in dev_result_files: 
    name =  ' '.join(list(map(str.capitalize, d.split('_')))[:3]) 
    with open(d, 'r') as f: 
        res = json.load(f) 
        results.update(
            {name: {
                'accuracy': res['metrics']['ACC']
                }
            }) 
df_results = pd.DataFrame(results)   
df_results

[]

## Clean up temporary folders

In [ ]:
if os.path.exists(ROOT_DIR):
    shutil.rmtree(ROOT_DIR, ignore_errors=True)